### tourAPI files

In [ ]:
## tourAPI - consolidate multiple split files in one file

import os

from utils import integration_xls

base_dir = "C:/Users/TAKO/Desktop/yanolja/data/raw_dataset/TourAPI"
directory_list = [name for name in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, name))]

for directory_ in directory_list:
    integration_xls((os.path.join(base_dir, directory_)).replace('\\', '/'))


### aihub - KVQA

In [ ]:
## aihub (KVQA) - many files to 1 csv file

import os
import json
import pandas as pd
from multiprocessing import Pool, Manager, cpu_count
from tqdm.notebook import tqdm
import chardet

base_directory = '/home/hwang/projects/yanolja/data/raw_dataset/aihub/KVQA'
output_path = '/home/hwang/projects/yanolja/data/dataset/aihub/KVQA/KVQA.csv'

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

def process_file(args):
    filename, file_path, skip_list = args
    
    encoding = detect_encoding(file_path)
    
    try:
        with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
            json_data = json.load(file)
        
        result = {
            "file_name": filename,
            "content": json.dumps(json_data, ensure_ascii=False)
        }
    except (json.JSONDecodeError, UnicodeDecodeError) as e:
        skip_list.append(filename)
        return None
    
    return result

def main():
    files = []
    for root, _, filenames in os.walk(base_directory):
        for filename in filenames:
            if filename.endswith('.json'):
                file_path = os.path.join(root, filename)
                files.append((filename, file_path))

    num_files = len(files)

    manager = Manager()
    skip_list = manager.list() 
    
    data = []
    with Pool(processes=cpu_count()) as pool:
        with tqdm(total=num_files, desc="Processing JSON files") as progress_bar:
            for result in pool.imap_unordered(process_file, [(filename, file_path, skip_list) for filename, file_path in files]):
                if result is not None:
                    data.append(result)
                progress_bar.update(1)
    
    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n총 {len(skip_list)}개의 파일이 스킵되었습니다.")
    if skip_list:
        print("스킵된 파일 목록:")
        for skipped_file in skip_list:
            print(f"- {skipped_file}")

main()


Processing JSON files:   0%|          | 0/153762 [00:00<?, ?it/s]


총 0개의 파일이 스킵되었습니다.


In [13]:
import pandas as pd

qa = pd.read_csv("/home/hwang/projects/yanolja/data/dataset/aihub/KVQA/KVQA.csv")

In [19]:
len(qa)

153762

In [33]:
seoul_file_name = []; seoul_content = []
for i in range(len(qa)):
    tmp_file_nmae = qa.iloc[i]['file_name']
    tmp_content = qa.iloc[i]['content']

    if json.loads(tmp_content)['data']['area'] == '서울':
        seoul_file_name.append(tmp_file_nmae)
        seoul_content.append(tmp_content)

In [35]:
import pandas as pd

seoul_qa = pd.DataFrame({'file_name': seoul_file_name, 'content': seoul_content})

seoul_qa.head(3)

,file_name,content
0,qa_66137_0_su_3_핫트랙스 디큐브시티점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6..."
1,qa_696160_0_su_3_유니클로 현대백화점 목동점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6..."
2,qa_652456_0_su_3_ABC마트 ST 대학로점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6..."


In [59]:
seoul_qa.iloc[0]['content']

'{"data": {"dataset": "2-083-219", "POI_id": "66137", "travelType": "쇼핑", "place": "핫트랙스 디큐브시티점", "area": "서울"}, "imageInfo": [{"image_id": 0, "filename": "66137_0_su_3_핫트랙스 디큐브시티점_1_001.png", "mapx": 37.5081230677129, "mapy": 126.888212953215, "width": 3840, "height": 2160, "caption_info": ["핫트랙스 디큐브시티점(신도림동)", "간판"]}, {"image_id": 1, "filename": "66137_0_su_3_핫트랙스 디큐브시티점_1_002.png", "mapx": 37.5081230677129, "mapy": 126.888212953215, "width": 3840, "height": 2160, "caption_info": ["핫트랙스 디큐브시티점(신도림동)", "건물"]}, {"image_id": 2, "filename": "66137_0_su_3_핫트랙스 디큐브시티점_1_003.png", "mapx": 37.5081230677129, "mapy": 126.888212953215, "width": 3840, "height": 2160, "caption_info": ["핫트랙스 디큐브시티점(신도림동)", "진열대"]}, {"image_id": 3, "filename": "66137_0_su_3_핫트랙스 디큐브시티점_1_004.png", "mapx": 37.5081230677129, "mapy": 126.888212953215, "width": 3840, "height": 2160, "caption_info": ["핫트랙스 디큐브시티점(신도림동)", "진열대"]}, {"image_id": 4, "filename": "66137_0_su_3_핫트랙스 디큐브시티점_1_005.png", "mapx": 37.5081230677129,

In [43]:
place_list = []

for i in range(len(seoul_qa)):
    place_list.append(json.loads(seoul_qa.iloc[i]['content'])['data']['place'])

In [44]:
place_list[:5]

['핫트랙스 디큐브시티점', '유니클로 현대백화점 목동점', 'ABC마트 ST 대학로점', '에잇세컨즈 명동점', '신원시장']

### aihub - corpus

In [ ]:
## aihub (corpus) - many files to 1 csv file 

import os
import json
import pandas as pd
from multiprocessing import Pool, Manager, cpu_count
from tqdm.notebook import tqdm
import chardet

directory_path = '/home/hwang/projects/yanolja/data/raw_dataset/aihub/corpus'
output_path = '/home/hwang/projects/yanolja/data/dataset/aihub/corpus/corpus.csv'

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

def process_file(args):
    filename, queue, skip_list = args
    file_path = os.path.join(directory_path, filename)
    
    encoding = detect_encoding(file_path)
    
    try:
        with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
            json_data = json.load(file)
        
        result = {
            "file_name": filename,
            "content": json.dumps(json_data, ensure_ascii=False)
        }
    except (json.JSONDecodeError, UnicodeDecodeError) as e:
        skip_list.append(filename)
        return None
    
    queue.put(1)
    return result

def main():
    files = [f for f in os.listdir(directory_path) if f.endswith('.json') or f.endswith('.txt')]
    num_files = len(files)
    
    manager = Manager()
    queue = manager.Queue()
    skip_list = manager.list() 
    
    data = []
    with Pool(processes=cpu_count()) as pool:
        with tqdm(total=num_files, desc="Processing JSON files") as progress_bar:
            for result in pool.imap_unordered(process_file, [(filename, queue, skip_list) for filename in files]):
                if result is not None:
                    data.append(result)
                progress_bar.update(1)
    
    df = pd.DataFrame(data)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n총 {len(skip_list)}개의 파일이 스킵되었습니다.")
    if skip_list:
        print("스킵된 파일 목록:")
        for skipped_file in skip_list:
            print(f"- {skipped_file}")

main()


In [ ]:
import pandas as pd

corpus = pd.read_csv("/home/hwang/projects/yanolja/data/dataset/aihub/corpus/corpus.csv")

In [ ]:
print(len(corpus))

In [51]:
corpus.head()

,file_name,content
0,관광 콘텐츠_관광지_자연관광_온라인_네이버_대부잠수교_E_48_231016.json,"{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
1,관광 콘텐츠_관광지_문화관광_온라인_한국민족문화대백과사전_고창북중학교_W_3_230...,"{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
2,관광 콘텐츠_관광지_역사관광_온라인_기타_봉일사지삼층석탑_S_4_231011.json,"{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
3,관광 콘텐츠_관광지_역사관광_온라인_네이버_동복향교_W_54_231030.json,"{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
4,관광 콘텐츠_관광지_문화관광_온라인_네이버_부천문화원_S_20_231006.json,"{""info"": {""creator"": ""세명소프트"", ""description"": ""..."


In [58]:
print(corpus.iloc[0]['file_name'])
print(corpus.iloc[0]['content'])

print(corpus.iloc[2]['file_name'])
print(corpus.iloc[2]['content'])

관광 콘텐츠_관광지_자연관광_온라인_네이버_대부잠수교_E_48_231016.json
{"info": {"creator": "세명소프트", "description": "관광 특화 말뭉치"}, "tour_info": {"file_name": "관광 콘텐츠_관광지_자연관광_온라인_네이버_대부잠수교_E_48_231016.csv", "Main_cate": "관광 콘텐츠", "2nd_cate": "관광지", "3rd_cate": "자연관광", "method": "온라인", "source": "네이버", "Tourist Spot": "대부잠수교", "location": "E", "docu_num": "48", "date": "231016"}, "docu_info": {"content": "대부잠수교", "contains": "관광지명 대부잠수교 개요 대구에서 경산으로 향하는 길에 자리한 대부 잠수교는 아름다운 산책로로 유명하다 이곳은 계절마다 다른 매력을 선사한다 봄이면 파릇파릇한 보리 싹들이 마음을 설레게 하고, 싱그러운 청보리가 맞아준다 강변을 따라 바람과 청보리가 만드는 화음은 마음을 맑게 한다 여름에는 황금빛 해바라기가 물결을 이루며 고흐의 그림 같은 풍경을 연상시킨다 주소(AD) 경상북도 경산시 하양읍 대조리 전화번호(TE) 053-810-5363 부대정보(UN) 주차가능", "count": "51", "sentences": [{"sentcenceId": "0001", "sentence": "관광지명 대부잠수교", "annotations": [{"TagText": "대부잠수교", "TagId": "001", "Tagclass": "O", "TagCode": "AF", "startPos": 5, "endPos": 9}], "sentence_Tagclass": ""}, {"sentcenceId": "0002", "sentence": "개요 대구에서 경산으로 향하는 길에 자리한 대부 잠수교는 아름다운 산책로로 유명하다", "annotations": [{"TagText"

In [ ]:
json.loads(corpus['content'])

0         {"info": {"creator": "세명소프트", "description": "...
1         {"info": {"creator": "세명소프트", "description": "...
2         {"info": {"creator": "세명소프트", "description": "...
3         {"info": {"creator": "세명소프트", "description": "...
4         {"info": {"creator": "세명소프트", "description": "...
                                ...                        
812635    {"info": {"creator": "세명소프트", "description": "...
812636    {"info": {"creator": "세명소프트", "description": "...
812637    {"info": {"creator": "세명소프트", "description": "...
812638    {"info": {"creator": "세명소프트", "description": "...
812639    {"info": {"creator": "세명소프트", "description": "...
Name: content, Length: 812640, dtype: object

In [71]:

corpus_dict = dict(zip(
    corpus['content'].apply(lambda x: json.loads(x).get('tour_info', {}).get('Tourist Spot')),
    corpus['content']
))

seoul_corpus = [corpus_dict.get(place, None) for place in place_list]

### QA, corpus integration  (seoul)

In [ ]:
seoul_qa_corpus = pd.DataFrame({'place': seoul_qa['file_name'], 'qa': seoul_qa['content'], 'corpus': seoul_corpus})

In [84]:
seoul_qa_corpus.head()

,place,qa,corpus
0,qa_66137_0_su_3_핫트랙스 디큐브시티점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6...",None
1,qa_696160_0_su_3_유니클로 현대백화점 목동점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6...","{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
2,qa_652456_0_su_3_ABC마트 ST 대학로점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6...","{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
3,qa_652515_0_su_3_에잇세컨즈 명동점_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""6...","{""info"": {""creator"": ""세명소프트"", ""description"": ""..."
4,qa_44629_0_su_3_신원시장_1.json,"{""data"": {""dataset"": ""2-083-219"", ""POI_id"": ""4...",None


In [85]:
seoul = seoul_qa_corpus[seoul_qa_corpus['corpus'].notna()]

print(len(seoul))

2956


### make instruction dataset using GPT API

In [ ]:
## raw dataset -> instruction dataset 
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_tourism_qa(location_name, description):
    
    prompt = f"""
    당신이 전문 AI 여행 도우미라고 가정해 보겠습니다. 아래 제공된 정보를 바탕으로 이 관광지에 대한 논리적인 질문과 그 독특한 측면과 방문객의 매력을 설명하는 자세한 답변을 사실에 기반하여 작성하세요.

    관광지: {location_name}
    관광지 개요: {description}

    생성:
    - 질문: 컨설팅 어조로, 이 관광지에 대해 관광지 개요를 참고하여 1~3 문장의 질문 3가지를 작성하세요.
    - 답변: 각 질문에 대해서 관광지 개요를 참고하여 해당 관광지의 매력, 분위기, 역사, 특징을 자세히 설명하는 8~12개의 문장으로 답하세요. 독자에게 친숙하고 접근하기 쉽고 포괄적인 내용을 유지하며 반드시 사실에 기반하여 답변해야 합니다..
    """

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 전문 AI 여행 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )

    question_answer = response.choices[0].message.content
    return question_answer

In [ ]:
## raw dataset => instruction dataset

import pandas as pd

data = pd.read_excel(r'C:\Users\TAKO\Desktop\yanolja\data\dataset\tourAPI\touristAttractions.xls')
data = data[0:500]

qa_pairs = []
for ind, row in data.iterrows():
    location_name = row['명칭']
    description = row['개요']

    qa = generate_tourism_qa(location_name, description)

    qa_pairs.append({
        'touristAttractions': location_name,
        'QA': qa,
    })


qa_df = pd.DataFrame(qa_pairs)
qa_df.to_csv(r'C:\Users\TAKO\Desktop\yanolja\data\instruction_dataset\tourAPI\qa_touristattr_name_description_gpt4o_mini_1106.csv', index=False)

In [59]:
qa_df.iloc[0]["QA"].split("###")[1:]

[' 질문 1:\n수성못 유원지의 역사적 배경은 무엇이며, 이곳의 조성 과정에서 어떤 중요한 사건이 있었나요?\n\n',
 ' 답변:\n수성못 유원지는 1925년에 조성된 인공못으로, 일제강점기에 농업용수 공급을 위해 만들어졌습니다. 이 시기는 한국 역사에서 중요한 전환점으로, 일본의 식민지 지배 아래에서 다양한 사회 기반 시설이 형성되었습니다. 수성못은 당시 농업의 효율성을 높이기 위한 중요한 수자원으로 기능했으며, 지금은 그 역할이 크게 변모하여 지역 주민과 관광객들에게 여가와 휴식을 제공하는 공간으로 자리잡았습니다. 수성못의 조성 과정에서 지역 사회의 필요와 자연 환경이 조화를 이루며, 현대에 이르러서는 수변 휴식공간으로 진화해왔습니다. 그 결과, 수성못은 단순한 저수지를 넘어 자연과 함께하는 명소로 발전했습니다.\n\n',
 ' 질문 2:\n수성못 유원지에서 제공하는 놀이시설과 활동은 어떤 것들이 있으며, 가족 단위 방문객에게 어떤 매력을 제공하나요?\n\n',
 ' 답변:\n수성못 유원지는 다양한 놀이시설과 활동으로 가족 단위 방문객들에게 매력을 제공합니다. 수성랜드는 유람선, 바이킹, 범퍼카, 회전목마 등 다양한 놀이기구를 갖추고 있어 어린이와 어른 모두가 즐길 수 있는 공간입니다. 또한, 수성못에서의 보트놀이와 오리배 타기는 가족들이 함께 시간을 보내기에 최적의 활동입니다. 이곳의 자연경관은 아이들에게는 신나는 모험의 공간이 되고, 부모님들에게는 편안한 휴식처가 됩니다. 두산폭포와 같은 볼거리는 시각적으로도 즐거움을 주며, 주변의 산책로는 온 가족이 함께 산책하며 대화를 나누기에 좋은 환경을 제공합니다. 이러한 다양한 시설과 활동으로 인해 수성못 유원지는 가족 나들이에 적합한 장소로 알려져 있습니다.\n\n',
 ' 질문 3:\n수성못 미디어아트 음악분수는 어떤 특별한 경험을 제공하며, 방문객들에게 어떤 감동을 줄 수 있나요?\n\n',
 ' 답변:\n수성못 미디어아트 음악분수는 독특한 시각 및 청각적 경험을 제공하는 매력적인 명소입니다. 이 음악

In [ ]:
## 